# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов оператора сотовой связи, которые уже перешли на тарифы "Ультра" и "Смарт". Задача – построить модель классификации, которая выберет подходящий тариф клиенту. Минимальная требуемая точность модели (accuracy) 0.75

## Откроем и изучим файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
import warnings
warnings.simplefilter('ignore')

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


В таблице 3214 наблюдений и 5 признаков. Предобработка данных была выполнена ранее, поэтому переходим к подготовки модели

## Разобьём данные на выборки

In [4]:
# В соотношении 60%-20%-20% разобьём наш датасет на тренировочную, валидационную и тестовые выборки соответственно
# Установим стратификацию, поскольку в целевом признаке может быть большая количественная разница между категориальными значениями
df_train, df_test = train_test_split(df, test_size=0.4, stratify=df['is_ultra'], random_state=1)
df_valid, df_test = train_test_split(df_test, test_size=0.5, stratify=df_test['is_ultra'], random_state=1)

features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']

In [5]:
# Проверим разбивку
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)

(1928, 4)
(643, 4)
(643, 4)


## Исследуем модели

Ниже мы будем использовать функцию *GridSearchCV* для подбора наиболее эффективных гиперпараметров каждой модели

In [6]:
# Обучим "Случайный лес"

In [7]:
%%time
forest_model = RandomForestClassifier(random_state=1234)

forest_param_grid = {'n_estimators': range (10, 51, 10),
              'max_depth': range (1, 11),
              'min_samples_leaf': range (1, 11),
              'min_samples_split': range (1, 11) }

forest_gs = GridSearchCV(forest_model, forest_param_grid, scoring='accuracy')

forest_gs.fit(features_train, target_train)

CPU times: user 26min 17s, sys: 24.8 s, total: 26min 42s
Wall time: 26min 44s


GridSearchCV(estimator=RandomForestClassifier(random_state=1234),
             param_grid={'max_depth': range(1, 11),
                         'min_samples_leaf': range(1, 11),
                         'min_samples_split': range(1, 11),
                         'n_estimators': range(10, 51, 10)},
             scoring='accuracy')

In [8]:
# Выведем лучший показатель accuracy и наиболее эффективные аргументы гиперпараметров
print(f'Accuracy наилучшей модели: {forest_gs.best_score_}')
print(f'Гиперпараметры наилучшей модели: {forest_gs.best_params_}')

Accuracy наилучшей модели: 0.81223336249243
Гиперпараметры наилучшей модели: {'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 9, 'n_estimators': 50}


In [9]:
# Запишем лучшую версию модели в переменную
best_forest = forest_gs.best_estimator_

In [10]:
# Обучим "Дерево решений"
# Циклом подберём наиболее эффективную глубину "дерева"
# Запишем лучшую версию модели в переменную
best_tree = None
best_result = 0
for depth in range(1, 11):
    tree_model = DecisionTreeClassifier(random_state=1234, max_depth=depth)
    tree_model.fit(features_train, target_train)
    predictions = tree_model.predict(features_valid)
    result = accuracy_score(target_valid, predictions)
    if result > best_result:
        best_tree = tree_model
        best_result = result
        
print("Accuracy наилучшей модели:", best_result)        

Accuracy наилучшей модели: 0.7993779160186625


In [11]:
# Обучим логистическую регрессию
# Найдём лучшие аргументы гиперпараметров
log_model = LogisticRegression(random_state=1234)

log_param_grid = { 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                   'penalty':['none', 'elasticnet', 'l1', 'l2'],
                   'C':[0.001, 0.01, 0.1, 1, 10, 100] }

log_gs = GridSearchCV(log_model, log_param_grid, scoring='accuracy')

log_gs.fit(features_train, target_train)

GridSearchCV(estimator=LogisticRegression(random_state=1234),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'penalty': ['none', 'elasticnet', 'l1', 'l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='accuracy')

In [12]:
# Выведем лучшие accuracy и аргументы гиперпараметров
print(f'Accuracy наилучшей модели: {log_gs.best_score_}')
print(f'Гиперпараметры наилучшей модели: {log_gs.best_params_}')

Accuracy наилучшей модели: 0.748969786689994
Гиперпараметры наилучшей модели: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}


In [13]:
# Запишем лучшую версию модели в переменную
best_log = log_gs.best_estimator_

Итого у нас получились три модели со следующими показателями *accuracy*:
- RandomForestClassifier ~ 81
- DecisionTreeClassifier ~ 0.80
- LogisticRegression ~ 0.75

## Проверим модель на тестовой выборке

Поскольку лучший показатель accuracy получился у случайного леса, мы будем использовать именно эту модель для проверки точности предсказаний на тесте 

In [14]:
best_forest.predict(features_test)
predictions_forest = best_forest.predict(features_test)
result_forest = accuracy_score(target_test, predictions_forest)
print(f'Точность ответов: {result_forest}')

Точность ответов: 0.8273716951788491


## Проверим модели на адекватность

Для анализа адекватности наших моделей сравним их *accuracy* с *accuracy* модели-манекена, который поможет нам зафиксировать "наинвый прогноз", с точки зрения частоты выпада категориальных значений

In [15]:
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=12345)

dummy_clf.fit(features_train, target_train)

dummy_clf.score(features_test, target_test)

0.6936236391912908

Accuracy манекена значительно ниже accuracy наших моделей, а значит, они отвечают требованию адекватности

<div class="alert alert-block alert-success"> <b> Комментарий ревьюера ✔️</b>

Абсолютно точно!
</div>


<div class="alert alert-block alert-warning"> <b> Комментарий ревьюера ⚠️</b> 
    
Смотри, в реальной практике заказчику очень важно видеть четкий и структурированный вывод, о том, что сделанно в проекте. Бывает, что заказчик даже не будет смотреть код, а сразу будет читать вывод исследования. Поэтому в проектах нужно писать расширенный вывод. 
    
</div>   

<div  style="background-color: aliceblue; border-left: 7px solid blue; padding: 15px; ">
<b> <font color='blue' >   Ревью: заключительный комментарий </font></b> 
    
Иван, получился хороший проект. Верно разделяешь выборки и готовишь данные к обучению. Вижу твоё понимание и заинтересованность - это очень важно и круто! Очень не хватает рассуждений и выводов, очень надеюсь, что ты учтёшь это в будущих проетах. 
    
Ознакомься с моими комментариями - если остались или возникли вопросы дай знать. Если нет - проект будет принят

Желаю тебе успехов! 
    
Жду тетрадку обратно 🤩
  
Дополнительно, что можно почитать:     
1) Про этапы анализа данных: https://pythonru.com/baza-znanij/process-analiza-dannyh
    
2) «Статистика для всех», Сара Бослаф - это очень полезная книга про основные статистические методы, знание которых облегчит жизнь при анализе: https://www.ozon.ru/product/statistika-dlya-vseh-140032797/?sh=noGsf1nAbQ 

3) Справочник по визуализации данных, собраны все библиотеки по python с примерами: https://python-charts.com

4) Изучай статистику, она очень нужна, вот бесплатный небольшой курс: https://stepik.org/course/76/promo
    
5) Курсы бесплатные по ML введение: https://stepik.org/course/4852/syllabus
    
6) Канал на ютубе про математику, ML и не только: https://www.youtube.com/c/selfedu_rus/videos
</div> 

<div class="alert alert-info"> <b>Комментарий студента:</b>
    Благодарю тебя за столь полезные комментарии и кладезь знаний!) Ниже я добавил общий вывод по проделанной работе, чтобы заказчик сразу мог понять, какую модель лучше использовать</div>

## Общий вывод

Для предсказания наиболее подходящего тарифа каждому отдельному пользователю мы проанализировали работу трёх моделей на имеющейся выборке данных. Результаты точности представлены ниже:

- `RandomForestClassifier` ~ 81
- `DecisionTreeClassifier` ~ 0.80
- `LogisticRegression` ~ 0.75

Проверив наши модели на адекватность в сравнении с моделью-манекеном, мы пришли к выводу, что все они действительно эффективны, а значит, для предсказания тарифов мы можем с уверенностью использовать лучшую из выбранных нами моделей – `Случайный лес`


<div class="alert alert-block alert-success"> <b> Комментарий ревьюера v2 ✔️</b>

Отлично! 😌
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
